In [ ]:
import json
import sys
import os

def clean_notebook(filepath):
    try:
        filepath = os.path.abspath(filepath)
        with open(filepath, "r", encoding="utf-8") as f:
            notebook = json.load(f)
        modified = False
        if "metadata" in notebook and "widgets" in notebook["metadata"]:
            del notebook["metadata"]["widgets"]
            modified = True
        if modified:
            with open(filepath, "w", encoding="utf-8") as f:
                json.dump(notebook, f, indent=2)
            print(f"Cleaned {filepath}")
            return True
        return False
    except Exception as e:
        print(f"Error processing {filepath}: {str(e)}", file=sys.stderr)
        return False

if __name__ == "__main__":
    for path in sys.argv[1:]:
        modified_notebook = clean_notebook(path)
        if modified_notebook:
            print(f"Modified {path}")
        else:
            print(f"No changes to {path}")

In [ ]:
!pip install -q gradio transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Step 2: Basic setup and GPU check
import os
import torch
from google.colab import output
print("Checking GPU availability...")
!nvidia-smi
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


Checking GPU availability...
Sat May 17 16:22:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+------------------

In [ ]:
# Step 3: Import libraries
import gradio as gr
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# Step 4: Define model map (models that work well on Colab's T4 GPU)
model_map = {
    "GPT-2": "gpt2",
    "DistilGPT-2": "distilgpt2",
    "T5-small": "google/flan-t5-small",
    "OPT-350M": "facebook/opt-350m",
    "BLOOM-560M": "bigscience/bloom-560m",
}


In [ ]:
# Step 5: Create model loader with caching
loaded_models = {}

def load_pipeline(model_name):
    """Load a model pipeline with caching"""
    if model_name in loaded_models:
        return loaded_models[model_name]

    print(f"Loading model: {model_name}")
    model_id = model_map[model_name]

    # Use more memory-efficient loading options
    if model_id.startswith("google/flan-t5"):
        pipe = pipeline(
            "text2text-generation",
            model=model_id,
            device_map="auto" if device == "cuda" else "cpu",
        )
    else:
        # For autoregressive models
        pipe = pipeline(
            "text-generation",
            model=model_id,
            device_map="auto" if device == "cuda" else "cpu",
        )

    loaded_models[model_name] = pipe
    return pipe

In [ ]:
# Step 6: Define prompt formatting function
def format_prompt(model_name, topic, position, opponent_text=None):
    """Format prompts according to each model's preferred format"""
    base_prompt = f"Debate topic: {topic}. You are arguing {'FOR' if position == 'A' else 'AGAINST'} the position."

    if opponent_text:
        base_prompt += f" Your opponent said: {opponent_text}. Provide a strong rebuttal."
    else:
        base_prompt += " Provide your opening argument with clear reasoning."

    # T5 models need special handling
    if model_name == "T5-small":
        return f"Summarize debate point: {base_prompt}"

    return base_prompt

In [ ]:
# Step 7: Define text generation function
def generate_text(pipe, prompt, model_name, max_new_tokens=150):
    """Generate text based on model type"""
    if model_name == "T5-small":
        result = pipe(prompt, max_length=max_new_tokens, do_sample=True, temperature=0.7)
        return result[0]['generated_text']
    else:
        result = pipe(prompt, max_new_tokens=max_new_tokens, do_sample=True, temperature=0.7)
        # Extract just the generated portion, not the full prompt+response
        generated_text = result[0]['generated_text']
        if prompt in generated_text:
            return generated_text[len(prompt):].strip()
        return generated_text.strip()


In [ ]:
# Step 8: Define debate turn function
def debate_turn(model_name, topic, position, opponent_text=None, max_words=150):
    """Generate one turn of debate"""
    pipe = load_pipeline(model_name)
    prompt = format_prompt(model_name, topic, position, opponent_text)
    return generate_text(pipe, prompt, model_name, max_words)

# Step 9: Define debate evaluation function
def evaluate_debate(judge_model, topic, transcript):
    """Evaluate the debate using a judge model"""
    pipe = load_pipeline(judge_model)

    judge_prompt = f"""
    Topic: {topic}

    Debate transcript:
    {transcript}

    As a neutral judge, evaluate both arguments on:
    1. Reasoning quality
    2. Persuasiveness
    3. Overall strength

    And declare a winner between Position A (FOR) and Position B (AGAINST).
    """

    if judge_model == "T5-small":
        judge_prompt = f"Summarize debate winner: {topic} debate. {transcript}"

    return generate_text(pipe, judge_prompt, judge_model, max_new_tokens=200)


In [ ]:
# Step 10: Define the full debate function
def run_debate(debater_a, debater_b, judge, topic, num_rounds, max_words):
    """Run a full debate with specified number of rounds"""
    transcript = []

    # Opening arguments
    a_response = debate_turn(debater_a, topic, "A", None, max_words)
    transcript.append(f"[Debater A - FOR - Opening]\n{a_response}")

    b_response = debate_turn(debater_b, topic, "B", a_response, max_words)
    transcript.append(f"[Debater B - AGAINST - Opening]\n{b_response}")

    # Additional rounds
    for round_num in range(2, num_rounds + 1):
        a_response = debate_turn(debater_a, topic, "A", b_response, max_words)
        transcript.append(f"[Debater A - FOR - Round {round_num}]\n{a_response}")

        b_response = debate_turn(debater_b, topic, "B", a_response, max_words)
        transcript.append(f"[Debater B - AGAINST - Round {round_num}]\n{b_response}")

    # Format full transcript
    full_transcript = "\n\n".join(transcript)

    # Get verdict
    verdict = evaluate_debate(judge, topic, full_transcript)

    return full_transcript, verdict

In [ ]:
# Step 11: Create Gradio interface
with gr.Blocks(title="AI Debate Platform") as demo:
    gr.Markdown("## 🤖 AI Debate Platform for Google Colab")
    gr.Markdown("This app lets you pit different language models against each other in a structured debate.")

    with gr.Row():
        with gr.Column():
            debater_a = gr.Dropdown(
                list(model_map.keys()),
                label="Debater A (FOR)",
                value="GPT-2"
            )
        with gr.Column():
            debater_b = gr.Dropdown(
                list(model_map.keys()),
                label="Debater B (AGAINST)",
                value="BLOOM-560M"
            )
        with gr.Column():
            judge = gr.Dropdown(
                list(model_map.keys()),
                label="Judge Model",
                value="DistilGPT-2"
            )

    with gr.Row():
        topic = gr.Textbox(
            label="Debate Topic",
            value="Is artificial intelligence beneficial for humanity?",
            lines=2
        )

    with gr.Row():
        with gr.Column():
            num_rounds = gr.Slider(
                minimum=1,
                maximum=3,
                value=2,
                step=1,
                label="Number of Debate Rounds"
            )
        with gr.Column():
            max_words = gr.Slider(
                minimum=50,
                maximum=200,
                value=100,
                step=10,
                label="Max Words per Response"
            )

    start_btn = gr.Button("Start Debate", variant="primary")

    with gr.Row():
        transcript = gr.Textbox(label="🗣️ Debate Transcript", lines=15)

    with gr.Row():
        verdict = gr.Textbox(label="⚖️ Judge's Verdict", lines=5)

    # Link button to function
    start_btn.click(
        fn=run_debate,
        inputs=[debater_a, debater_b, judge, topic, num_rounds, max_words],
        outputs=[transcript, verdict]
    )

    gr.Markdown("""
    ### Notes for Colab Users:
    - This implementation uses smaller models that work well on Colab's free T4 GPU
    - Each model uses ~2-4GB of VRAM, so you can run 3 different models on a 16GB Colab GPU
    - First model load will take a few minutes, subsequent runs will be faster
    - If you encounter CUDA out of memory errors, try reducing the number of rounds or using fewer/smaller models
    """)

In [ ]:
# Step 12: Launch the app in Colab-friendly way
demo.queue().launch(share=True, debug=True)

# Display helpful message for Colab
print("\n" + "="*50)
print("IMPORTANT: In Colab, click on the public URL above to access the interface")
print("The interface will open in a new tab. When done, return to this notebook and stop the cell.")
print("="*50)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8cfedc1f6b40395d28.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Loading model: GPT-2


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cuda:0
